In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
data = pd.read_csv('nyc-collisions.csv')
data.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,number_of_persons_injured,...,vehicle_type_code1,contributing_factor_vehicle_2,vehicle_type_code2,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,cross_street_name,contributing_factor_vehicle_5,vehicle_type_code_5
0,2024-12-06T00:00:00.000,14:16,QUEENS,11367.0,40.742200,-73.829880,"{'latitude': '40.7422', 'longitude': '-73.8298...",61 RD,136 ST,0,...,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-12-06T00:00:00.000,9:20,BRONX,10475.0,40.890076,-73.819850,"{'latitude': '40.890076', 'longitude': '-73.81...",BOSTON RD,ROPES AVE,1,...,Sedan,Unspecified,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-06T00:00:00.000,6:10,BROOKLYN,11221.0,40.686367,-73.929750,"{'latitude': '40.686367', 'longitude': '-73.92...",MALCOLM X BLVD,PUTNAM AVE,2,...,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN
3,2024-12-06T00:00:00.000,16:30,BROOKLYN,11214.0,40.608000,-74.001720,"{'latitude': '40.608', 'longitude': '-74.00172...",18 AVE,NEW UTRECHT AVE,0,...,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified,Bike,NaN,NaN,NaN
4,2024-12-06T00:00:00.000,16:48,QUEENS,11421.0,40.697560,-73.852745,"{'latitude': '40.69756', 'longitude': '-73.852...",WOODHAVEN BLVD,PARK LN S,1,...,E-Bike,Unspecified,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# since 'location' column contains the dictionary of location whcih is already stated in other columns, we will remove it from our dataframe for more memory-efficient of work.
data.drop(['location'],axis=1,inplace=True)

In [7]:
data.shape

(285147, 28)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285147 entries, 0 to 285146
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   crash_date                     285147 non-null  object 
 1   crash_time                     285147 non-null  object 
 2   borough                        194401 non-null  object 
 3   zip_code                       194363 non-null  float64
 4   latitude                       261920 non-null  float64
 5   longitude                      261920 non-null  float64
 6   on_street_name                 206104 non-null  object 
 7   off_street_name                138697 non-null  object 
 8   number_of_persons_injured      285147 non-null  int64  
 9   number_of_persons_killed       285147 non-null  int64  
 10  number_of_pedestrians_injured  285147 non-null  int64  
 11  number_of_pedestrians_killed   285147 non-null  int64  
 12  number_of_cyclist_injured     